In [2]:

# The higher the fitness score, the better the solution (Our goal is to maximize fitness value)

import collections
import csv
import random
from copy import deepcopy
from numpy import random
from numpy.random import randint


room_names = ['C301', 'C302', 'C303', 'C304', 'C305', 'C306', 'C307', 'C308', 'C309', 'C310','C311','C312','C313','C314','C315']
total_days = ['Thursday', 'Friday','Saturday', 'Sunday', 'Monday']
classrooms = collections.namedtuple('classroom', 'room_name morning invig_morning noon invig_noon')


#------------------------------------------------------------------------------------------------------------------------
#                                                Some classes to keep Record 
#------------------------------------------------------------------------------------------------------------------------

# This class will basically contain the name of students alongwith the array containing the courses he studies
class StudentData:
    def __init__(self, name, courses):
        self.name = name
        self.courses = []
        self.courses.append(courses)

    def add_course(self, course):
        self.courses.append(course)

    def __repr__(self):                                                # Returns string representation
        output = "Name: " + self.name + "\t" + "Courses: " + str(self.courses) + "\n"
        return output


def add_student(student_list, name, cc):                               # Add a course for a specific student in the list

    for student in student_list:
        if student.name == name:
            if cc not in student.courses:                              # To cater repeated value
                student.add_course(cc)
            return

    new_student = StudentData(name, cc)
    student_list.append(new_student)


# This class will basically act as a solution.It consists of days and a collection of room info, timee slots and
# teachers on invigilation duties. We have created a dictionary to store a collection corresponding each day of the week
class Schedule:
    def __init__(self, days=dict(), fitness=0):
        self.days = days
        self.fitness = fitness

        for day in total_days:
            self.days[day] = []



#------------------------------------------------------------------------------------------------------------------------
#                                                 Reading Data
#------------------------------------------------------------------------------------------------------------------------

def is_course_in_list(courses, code):
    for course in courses:
        if code == course[0]:
            return True

    return False


# Loading each csv in proper format for the further use 
def read_data():
    courses = []

    #with open("data/temp_courses.csv") as csv_file:       # Sample Dataset 
    with open("data/courses.csv") as csv_file:            # Provided Dataset
        csv_reader = csv.reader(csv_file, delimiter=',')  # Loading courses
        for row in csv_reader:
            code_title = row[0], row[1]

            if not is_course_in_list(courses, code_title[0]):
                courses.append(code_title)

    # -------------------- #

    teacher = []
    
    #with open("data/temp_teachers.csv") as csv_file:      # Sample Dataset
    with open("data/teachers.csv") as csv_file:           # Provided Dataset
        csv_reader = csv.reader(csv_file, delimiter=',')  # Loading teachers
        for row in csv_reader:
            if len(row) > 0:
                name = row[0]
                teacher.append(name)

    # -------------------- #

    student_list = []
    
    #with open("data/temp_studentcourses.csv") as csv_file:# Sample Dataset
    with open("data/studentCourse.csv") as csv_file:      # Provided Dataset
        csv_reader = csv.reader(csv_file, delimiter=',')  # Reading students and their courses
        line = 0
        for row in csv_reader:
            if line != 0:
                name = row[1]
                cc = row[2]
                add_student(student_list, name, cc)
            line += 1

    return courses, teacher, student_list                 # Returning all data




#------------------------------------------------------------------------------------------------------------------------
#                                                  Printing FUNCTIONS
#------------------------------------------------------------------------------------------------------------------------

def print_classes_list(classes_list):
    for _class in classes_list:
        morning_exam_code = _class.morning.ljust(8) if _class.morning else 'N/A'.ljust(8)
        morning_invigilator = _class.invig_morning.ljust(20) if _class.invig_morning else 'N/A'.ljust(20)
        noon_exam_code = _class.noon.ljust(8) if _class.noon else 'N/A'.ljust(8)
        noon_invigilator = _class.invig_noon.ljust(20) if _class.invig_noon else 'N/A'.ljust(20)

        print("ClassRoom:", _class.room_name)
        print("MorningExamCode:", morning_exam_code, "| MorningInvigilator:", morning_invigilator)
        print("NoonExamCode:", noon_exam_code, "| NoonInvigilator:", noon_invigilator)
        print()
    print("-----------------------------------------------------------")   



def print_schedule(schedule):
    for day in schedule.days:
        if day == 'Saturday' or day == 'Sunday':
            print("\nDay: ", day)
            print("NO EXAM TODAY\n")
            print("-----------------------------------------------------------")
        elif len(schedule.days[day]) > 0:
            print("\nDay: ", day)
            print_classes_list(schedule.days[day])


# def print_population(population):
#     count = 1
#     for schedule in population:

#         print("Schedule #", count, "\n")
#         print("Fitness: ", schedule.fitness)
#         for day in schedule.days:
#             print("\nDay: ", day, "\n\n")
#             print_classes_list(schedule.days[day])
#             print("\n")

#         count = count + 1

        
#------------------------------------------------------------------------------------------------------------------------
#                                                  Generating Populations
#------------------------------------------------------------------------------------------------------------------------

#function to generate set of random solutions.

def generate_population(population_size, courses, teachers):
    new_pop = []

    for i in range(0, population_size):
        schedule = Schedule(dict())

        for day in total_days:

            visited_indexes = []
            total_classrooms = random.randint(0, len(room_names))  
                                                            # Classrooms = room_name, morning, invig_morning, noon, invig_noon

            for j in range(total_classrooms):

                index = random.randint(0, len(room_names))  # Generating that number of classrooms

                while index in visited_indexes:
                    index = random.randint(0, len(room_names))

                room = room_names[index]
                visited_indexes.append(index)

                index1 = random.randint(0, len(courses))    # Randomly picking morning course
                m_course = courses[index1][0]

                index2 = random.randint(0, len(teachers))   # Randomly picking invigilator for morning
                m_invig = teachers[index2]

                index3 = random.randint(0, len(courses))    # Randomly picking noon course
                n_course = courses[index3][0]

                index4 = random.randint(0, len(teachers))   # Randomly picking invigilator for noon
                n_invig = teachers[index4]

                                                            # Noinspection PyArgumentList
                schedule.days[day].append(
                    classrooms(
                        room_name=room,
                        morning=m_course,
                        invig_morning=m_invig,
                        noon=n_course,
                        invig_noon=n_invig
                    )
                )

        new_pop.append(schedule)

    return new_pop


#------------------------------------------------------------------------------------------------------------------------
#                                                     CONSTRAINTS FUNCTIONS
#------------------------------------------------------------------------------------------------------------------------

# Hard Const 1 = Exam scheduled for every course
def hconstraint_all_courses(schedule, courses):         

    bool_check = False

    exam_list = []
    total_exams = 0

    for day in schedule.days:                           # Getting list of all exams in the schedule week
        class_list = schedule.days[day]

        for _class in class_list:                       # Examining every classroom assigned
            m_exam = _class.morning
            n_exam = _class.noon
            total_exams += 2

            if m_exam not in exam_list:
                exam_list.append(m_exam)

            if n_exam not in exam_list:
                exam_list.append(n_exam)

    exam_codes = []                                     # List of all exams

    for course in courses:
        code = course[0]
        exam_codes.append(code)                         # Gets list of all exam codes of all courses

    missing = 0
    for code in exam_codes:
        if code not in exam_list:                       # Comparing the two lists
            missing =  missing + 1                                # Number of courses not listed

    if missing == 0:                                    # If the constraint is satisfied completely
        bool_check = True

    num = (1 / (1 + missing)) 

    return num, bool_check


# Hard Const 2 = Minimum 3 courses per student
def hconstraint_three_courses(course_allocation):       

    for student in course_allocation:
        if len(student.courses) < 3:                    # If a student is registered in less than 3 courses
            return False

    return True


# Hard Const 2.1 : A student cannot give more than 1 exam at a time.
def hconstraint_clashing_exams(schedule, course_allocation):  

    exam_list = []

    for day in schedule.days:                                 # Getting list of all exams in this schedule
        classes_list = schedule.days[day]

        for _class in classes_list:                           # Storing all exams for that day
            exam_list.append(_class.morning)
            exam_list.append(_class.noon)

    exam_counts = dict(collections.Counter(exam_list))

    clashes = 0
    student_names_clashes = []
    bool_check = False

    for day in schedule.days:                                 # Getting list of classrooms for every day
        classes_list = schedule.days[day]

        morning_list = []
        noon_list = []

        for _class in classes_list:                           # Getting lists of all exams on morning and noon
            morning_list.append(_class.morning)
            noon_list.append(_class.noon)

        for student in course_allocation:                     # Examining all students and their allocated courses
            course_list = student.courses
            name = student.name

            clash_flag = False                                # Checking if record clashes for a student

            count = 0
            for course in morning_list:
                if course in course_list:
                    if course in exam_counts.keys():
                        if exam_counts[course] == 1:          # Exam only at this specific time
                            count += 1                        # Counting clashes in morning for a student

            if name not in student_names_clashes:
                if count > 1:
                    clash_flag = True
                    clashes += 1
                    student_names_clashes.append(name)

            count = 0
            for course in noon_list:
                if course in course_list:
                    if course in exam_counts.keys():
                        if exam_counts[course] == 1:
                            count += 1

            if name not in student_names_clashes:
                if count > 1 and not clash_flag:
                    clashes += 1                              # Counting clashes in evening for a student
                    student_names_clashes.append(name)

    if clashes == 0:                                          # If the constraint is satisfied completely
        bool_check = True

    num = (1 / (1 + clashes))

    return num, bool_check


#ALREADY SATISFIED
# Hard Const 3 Exam will not be held on weekends.
# Hard Const 4 Each exam must be held between 9 am and 5 pm.

 # Hard Const 5.1 : No teacher clashes at the same time. It will automatically satisfy Each exam must be invigilated by a teacher.
def hconstraint_teachers_sametime(schedule):                 

    clashes = 0
    bool_check = False

    for day in schedule.days:
        classes_list = schedule.days[day]
        morning_list = []
        noon_list = []

        for _class in classes_list:                           # Getting lists of all teachers on morning and noon
            morning_list.append(_class.invig_morning)
            noon_list.append(_class.invig_noon)

        dup_count_morning = dict(collections.Counter(morning_list))  # Getting the number of duplicate entries
        dup_count_noon = dict(collections.Counter(noon_list))

        for value in dup_count_morning.values():
            if value > 1:
                clashes += 1                                         # Calculating the number of clashes

        for value in dup_count_noon.values():
            if value > 1:
                clashes += 1

    if clashes == 0:                                                 # If the constraint is satisfied completely
        bool_check = True

    num = (1 / (1 + clashes)) + 1 # increasing its fitness because this was nots atisfying gain and again 

    return num, bool_check


# Hard Const 6 : No teacher has duties in a row
def hconstraint_teacher_samerow(schedule):                           

    consecutive = 0
    bool_check = False
    for day in schedule.days:
        classes_list = schedule.days[day]
        morning_list = []
        noon_list = []

        for _class in classes_list:                                  # Getting lists of all teachers on morning and noon
            morning_list.append(_class.invig_morning)
            noon_list.append(_class.invig_noon)

        for teacher in morning_list:
            if teacher in noon_list:
                consecutive += 1                                     # Calculating the number of consecutive duties

    if consecutive == 0:                                             # If the contraint is satisfied completely
        bool_check = True

    num = (1 / (1 + consecutive))

    return num, bool_check



def hconstraint_course_scheduled_once(schedule, courses): # Hard Const : In one population One course must be scheduled once only
    # Assuming schedule.days is a dictionary where keys are days and values are lists of classes
    # Each class has a list of courses scheduled in morning and noon
    scheduled = []
    bool_check = False

    for day in schedule.days:
        classes_list = schedule.days[day]

        for _class in classes_list:
            # Assuming _class.morning and _class.noon are lists of course codes
            scheduled.extend(_class.morning)
            scheduled.extend(_class.noon)

    # Counting the occurrences of each course in the schedule
    dup_counts = dict(collections.Counter(scheduled))

    # Checking if each course is scheduled exactly once
    for course in courses:
        if course not in dup_counts or dup_counts[course] != 1:
            return 0, False # Course not scheduled or scheduled more than once

    # If all courses are scheduled exactly once, the constraint is fully satisfied
    return 1, True



# this function will try to mak ethe schedule in less days a spossible.
# if there are two consecutive in which exams are schedued they will be considered as empty days. 
#Readme file also contains this info
def sconstraint_less_days(schedule):
    empty_days = 0
    for day_index in range(len(total_days) - 1):
        current_day = total_days[day_index]
        next_day = total_days[day_index + 1]
        if schedule.days[current_day] and schedule.days[next_day]:
            empty_days += 1
    return empty_days, empty_days <= 7  # Returning a tuple with empty days count and constraint satisfaction




def sconstraint_no_consecutive_exams(schedule, course_allocation):
    consecutive_exams_count = 0
    bool_check = False

    for student in course_allocation:
        exam_list = []

        # Collecting all exams for the student
        for course in student.courses:
            for day in schedule.days.values():
                for _class in day:
                    if course in (_class.morning, _class.noon):
                        exam_list.append((_class.morning, _class.noon))

        # Checking for consecutive exams
        for i in range(len(exam_list) - 1):
            current_exam = exam_list[i]
            next_exam = exam_list[i + 1]
            if current_exam[1] == next_exam[0]:
                consecutive_exams_count += 1

    # If there are no consecutive exams, the constraint is satisfied
    if consecutive_exams_count == 0:
        bool_check = True

    num = (1 / (1 + consecutive_exams_count))

    return num, bool_check


def sconstraint_mg_before(schedule, course_allocation):
    violations = 0
    bool_check = False

    for student in course_allocation:
        mg_exams = []
        cs_exams = []

        # Collect all MG and CS exams for the student
        for course in student.courses:
            if 'MG' in course:
                mg_exams.append(course)
            elif 'CS' in course:
                cs_exams.append(course)

        # Check if there are both MG and CS exams for the student
        if mg_exams and cs_exams:
            # Flatten the list of exams for the student across all days
            all_exams = [exam for day in schedule.days.values() for _class in day for exam in (_class.morning, _class.noon)]

            # Check if any MG exam is scheduled after any CS exam
            for mg_exam in mg_exams:
                for cs_exam in cs_exams:
                    try:
                        if all_exams.index(mg_exam) > all_exams.index(cs_exam):
                            violations += 1
                            break # No need to check further for this student
                    except ValueError:
                        # Handling the case where the exam is not found in the list
                        pass

    # If there are no violations, the constraint is satisfied
    if violations == 0:
        bool_check = True

    # The fitness value is inversely proportional to the number of violations
    # This means a lower number of violations results in a higher fitness value
    num = (1 / (1 + violations))

    return num, bool_check


    

# if this function returns true that means all the constraints satisfied 
def constraints_satisfied_check(schedule, courses, course_allocation):
    hc1, hb1 = hconstraint_all_courses(schedule, courses)
    hc2, hb2 = hconstraint_clashing_exams(schedule, course_allocation)
    hc3, hb3 = hconstraint_teachers_sametime(schedule)
    hc4, hb4 = hconstraint_teacher_samerow(schedule)
    hc5, hb5 = hconstraint_course_scheduled_once(schedule, courses)
    _, sb2 = sconstraint_less_days(schedule)
    sc3, sb3 = sconstraint_mg_before(schedule, course_allocation)

    
    if hb1 and hb2 and hb3 and hb4:  

# soft constraint 1 is also already satisfied because i have set exam times only 9-12 or 2-5
# soft constrain 4 already satisfied because i have satisfied the hard constraint 5 which is that 
# teachers cannot invigilate two exams in a row so in this way half of the teachers will be free in one slot
# Now i need at least one soft constraint  from sb2 and sb3 must be satisfied
        if sb2 > 0 or sb3:
            return True

    return False


#Function to print that which constarints are satisfied and which are not
def constraints_satisfied_print(schedule, courses, course_allocation):
    hc1, hb1 = hconstraint_all_courses(schedule, courses)
    hc2, hb2 = hconstraint_clashing_exams(schedule, course_allocation)
    hc3, hb3 = hconstraint_teachers_sametime(schedule)
    hc4, hb4 = hconstraint_teacher_samerow(schedule)
    hc5, hb5 = hconstraint_course_scheduled_once(schedule, courses)
    sc2 , sb2 = sconstraint_no_consecutive_exams(schedule, course_allocation)
    my_sc,sc_days = sconstraint_less_days(schedule)
    sc3, sb3 = sconstraint_mg_before(schedule, course_allocation)

    print("\n---------------------------------\n")
    print("FITNESS VALUES :", "\nHard Constraint 1 fitness = ", hc1, "\nHard Constraint 2 fitness = ", hc2, "\nHard Constraint 3 fitness = ", hc3, "\nHard Constraint 4 fitness = ", hc4, "\nHard Constraint 5 fitness = ", hc4, "\nSoft Constraint LessDays ExamSCheduling 2 fitness = ", my_sc, "\nSoft Constraint 2 fitness = ", sc2 ,"\nSoft Constraint 3 fitness = ", sc3)
    print("\n-------------Hard constraints at Current Generation-------------: \n")

    print("An exam will be scheduled for each course: ", end='')
    if hb1:
        print('\u2705')
    else:
        print('\U0001F6AB')

    # this will be true because if its not true we cannot reach there  bcoz our first constraint check was this in GA function   
    print("A student is enrolled in at least 3 courses: " + '\u2705') # because already satified this in generate_algo function
        
    print("A student cannot give more than 1 exam at a time: ", end='')
    if hb2:
        print('\u2705')
    else:
        print('\U0001F6AB')
    
    #satisfies this in print_schedule function
    print("Exam will not be held on weekends: "+ '\u2705') 
    
    #I have set only 2 times for exam so below is always satisifed 
    print("Each exam must be held between 9 am and 5 pm: "+ '\u2705') 
    

    print("A teacher cannot invigilate two exams at the same time: ", end='')
    if hb3:
        print('\u2705')
    else:
        print('\U0001F6AB')


    print("A teacher cannot invigilate two exams in a row: ", end='')
    if hb4:
        print('\u2705')
    else:
        print('\U0001F6AB')


    print("\n\n-------------Soft constraints at Current Generation-------------: \n")

    print("All students and teachers shall be given a break on Friday from 1-2: " + '\u2705')


    print(
    "A student shall not give more than 1 exam consecutively: ", end='')      
    if sb2:
        print('\u2705')
    else:
        print('\U0001F6AB')
                    
          
    print(
        "If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held "
        "before their CS course exam: ",
        end='')

    if sb3:
        print('\u2705')
    else:
        print('\U0001F6AB')

          
    print("Half of faculty free at same time: " + '\u2705')
          
       
    print("Is Schedule in less days According to sconstraint_less_days() function: ", end='')
    if sc_days > 0:
        print('\u2705')
    else:
        print('\U0001F6AB')
    print()
    print()

#------------------------------------------------------------------------------------------------------------------------
#                                             Functions required  Genetic Algorithm
#------------------------------------------------------------------------------------------------------------------------
def calculate_fitness(population, courses, course_allocation):
    for schedule in population: # For each schedule in a population
        hc1, hb1 = hconstraint_all_courses(schedule, courses)
        hc2, hb2 = hconstraint_clashing_exams(schedule, course_allocation)
        hc3, hb3 = hconstraint_teachers_sametime(schedule)
        hc4, hb4 = hconstraint_teacher_samerow(schedule)
        hc5, hb5 = hconstraint_course_scheduled_once(schedule, courses) # New constraint for each course scheduled
        my_sc, empty_days = sconstraint_less_days(schedule)
        sc3, sb3 = sconstraint_mg_before(schedule, course_allocation)

        # Calculating the fitness of the schedule, emphasizing hard constraints
        fitness = hc1 * 2.5  + hc2 + hc3 + hc4 + hc5 + my_sc/2 + sc3/2
        
#         # Adjusting soft constraints to ensure they do not overshadow hard constraints
#         fitness = fitness - (0.5 * my_sc) - (0.2 * sc3) # Reduced weight for soft constraints
        schedule.fitness = fitness # Adding the fitness to the schedule

        if empty_days > 7: # Penalize schedules with too many empty days
            schedule.fitness = schedule.fitness - 1

    return population



def get_fitness(schedule):                                       # Returning fitness
    return schedule.fitness


def two_fittest_schedules(population):                           # Selecting two fittest solutions (schedules)
    pop = deepcopy(population)
    pop.sort(key=get_fitness, reverse=True)                      # Sorts in descending order
    return pop[0], pop[1]


def parent_selection(population):                                # Roulette Wheel Selection

    parents = []
    total_fitness = 0

    for schedule in population:
        total_fitness += schedule.fitness

    highest, second_highest = two_fittest_schedules(population)  # Getting two fittest solutions
    parents.append(highest)
    parents.append(second_highest)
    fitness_sum = 0

    while len(parents) < len(population):

        individual = random.randint(0, len(population))          # Getting a random index
        fitness_sum += population[individual].fitness
        if fitness_sum >= total_fitness:                         # Individual chosen based on its probability
            if population[individual] not in parents:
                parents.append(deepcopy(population[individual]))

    return parents


def mix_days(parent_a, parent_b):                                # Generating new schedule

    no_days_to_mix = randint(1, len(total_days))                 # Random crossover point
    child1 = Schedule()
    child2 = deepcopy(child1)

    i = 0
    for day in total_days:

        if i < no_days_to_mix:                                  # Taking that # of days from first parent
            classes_list_a = parent_a.days[day]
            classes_list_b = parent_b.days[day]

            child1.days[day] = deepcopy(classes_list_a)
            child2.days[day] = deepcopy(classes_list_b)

        else:                                                   # Taking rest of days from second parent
            classes_list_a = parent_a.days[day]
            classes_list_b = parent_b.days[day]

            child1.days[day] = deepcopy(classes_list_b)
            child2.days[day] = deepcopy(classes_list_a)

        i += 1

    return child1, child2


def mutate_schedule(schedule, mutation_probability, courses, teachers):  # Applying mutation on chromosomes

    if randint(0, 100) <= mutation_probability * 100:                    # Checking prob for mutation
        random_days = randint(0, len(total_days))                        # Selecting random no of days to change

        for i in range(0, random_days):
                                                                         # Choosing a random day
            idx = randint(0, len(total_days))
            day = total_days[idx]

            classes_list = schedule.days[day]

                                                                         # If it had assigned classes/exams
            if len(classes_list) > 0:
                if randint(0, 2) == 1:
                    for j in range(0, len(classes_list)):                # Will change one class at a time

                        if randint(0, 2) == 1:                           # 50% probabilty for mutation
                            index = random.randint(0, len(courses))      # Randomly replacing course
                            morning = courses[index][0]

                            index = random.randint(0, len(teachers))     # Randomly replacing invigilator
                            invig_morning = teachers[index]

                            index = random.randint(0, len(courses))      # Randomly replacing course
                            noon = courses[index][0]

                            index = random.randint(0, len(teachers))     # Randomly replacing invigilator
                            invig_noon = teachers[index]

                                                                         # Updating the values
                            classes_list[j] = classes_list[j]._replace(morning=morning, invig_morning=invig_morning)
                            classes_list[j] = classes_list[j]._replace(noon=noon, invig_noon=invig_noon)
                else:
                    classes_list.clear()

                                                       # If that day was empty, then assign it some classes/exams
            else:
                visited_indexes = []
                total_classrooms = random.randint(0,
                                                  5)   # Classrooms = room_name, morning, invig_morning, noon, invig_noon

                for j in range(total_classrooms):

                    index = random.randint(0, 9)       # Generating that number of classrooms

                    while index in visited_indexes:
                        index = random.randint(0, 9)

                    room = room_names[index]
                    visited_indexes.append(index)

                    index = random.randint(0, len(courses))   # Randomly picking morning course
                    m_course = courses[index][0]

                    index = random.randint(0, len(teachers))  # Randomly picking invigilator for morning
                    m_invig = teachers[index]

                    index = random.randint(0, len(courses))   # Randomly picking noon course
                    n_course = courses[index][0]

                    index = random.randint(0, len(teachers))  # Randomly picking invigilator for noon
                    n_invig = teachers[index]

                    classes_list.append(
                        classrooms(
                            room_name=room,
                            morning=m_course,
                            invig_morning=m_invig,
                            noon=n_course,
                            invig_noon=n_invig
                        )
                    )

    return schedule                                           # Returning the mutated schedule


def apply_crossover(population, crossover_probability):       # Applying crossover

    crossovered_population = []

                                                              # Equal length crossover
    while len(crossovered_population) < len(population):
        if randint(0, 100) <= crossover_probability * 100:
            parent_a, _ = two_fittest_schedules(population)   # Selecting fittest parent

                                                              # Selecting a random parent
            index_b = randint(0, len(population))
            parent_b = population[index_b]

            child1, child2 = mix_days(parent_a, parent_b)

            crossovered_population.append(deepcopy(child1))
            crossovered_population.append(deepcopy(child2))

    return crossovered_population


def apply_mutation(population, mutation_probability, courses, teachers):
    mutated_population = []

    for schedule in population:
        s = mutate_schedule(schedule, mutation_probability, courses, teachers)
        mutated_population.append(deepcopy(s))

    return mutated_population

#------------------------------------------------------------------------------------------------------------------------
#                                               Implementation of the Genetic Algorithm
#------------------------------------------------------------------------------------------------------------------------
def genetic_algo(population_size, max_generations, crossover_probability, mutation_probability, courses, teachers,
                 course_allocation):
    # if every student is not registered in three coirse we will quit here 
    if not hconstraint_three_courses(course_allocation):
        print("Failed Constraint that Every student must have at least 3 courses allocated. Try Again(")
        return None

    best_solution = None

                                              # Generating a list of schedules
    population = [generate_population(population_size, courses, teachers)]

    # For seeing if algorithm is unable to optimise further
    # stagant variable will keep track of the generation passed without any improvement in its fitness value. 
    # maily for check that if algorithm stucks at local optima 
    stagnant = 0  
    reset_count = 0
    solutions_list = []
    prev_best = None

    for i in range(max_generations):

                                              # Evaluating fitness
        pop = calculate_fitness(population[0], courses, course_allocation)

                                              # Selecting parents through roulette wheel selection
        parents = parent_selection(deepcopy(pop))

                                              # Applying crossover
        crossover_population = apply_crossover(parents, crossover_probability)
        calculate_fitness(crossover_population, courses, course_allocation)

                                              # Applying mutation
        mutated_population = apply_mutation(crossover_population, mutation_probability, courses, teachers)
        calculate_fitness(mutated_population, courses, course_allocation)

                                              # Finding fittest candidate
        schedule1, _ = two_fittest_schedules(mutated_population)

        if best_solution is None:
            stagnant = 0
            best_solution = deepcopy(schedule1)

        elif schedule1.fitness > best_solution.fitness:
            stagnant = 0
            best_solution = deepcopy(schedule1)

        if best_solution.fitness == prev_best:
            stagnant = stagnant + 1

        prev_best = deepcopy(best_solution.fitness)

        # THIS if Block will run only when constraints_satisfied_check(best_solution, courses, course_allocation) wil return trye
        # means when all constraints in this function returns true means we got a best solution
#         if constraints_satisfied_check(best_solution, courses, course_allocation):
#             constraints_satisfied_print(best_solution, courses, course_allocation)
#             print()
#             print(Back.BLACK + "                                                                         " + Back.RESET)
#             print(
#                 Back.BLACK + Fore.WHITE + "                             SOLUTION FOUND!!!!                          " + Fore.RESET + Back.RESET)
#             print(Back.BLACK + "                                                                         " + Back.RESET)
#             print()
#             return best_solution

        
        # No further optimisation for this # of generations
#         If the algorithm fails to find a better solution for a certain number of generations (in this case, 50),
#         so when stagant will be 50  that means 50 generations passed with same fitness values it indicates that 
#          the algorithm might be stuck in a local optimum so now we will start finding the best solution again
#          from the zero fitness value
        if stagnant == 50:
            if reset_count < 3:
                print(
                    "\n-------------------\nSearch space might not contain a significantly better solution so Algorithm is unable to optimise further. Starting over with a new random population.\n-------------------\n")
                solutions_list.append(deepcopy(best_solution))

                i = 0
                stagnant = 0
                reset_count += 1

                pop = generate_population(population_size, courses, teachers)
                best_solution = None
                population.clear()
                population.append(pop)
                continue
        # if reset_count > 3 that means more than three times stagant becomes 50 means
        # more than 3 times we had made new populations/more than 3 time our algorithm fails to find better solution 
            else:
                print(
                    "\nCannot optimise further.Terminating.. and Returning best solution upto now.\n")
                best, _ = two_fittest_schedules(solutions_list)
                return best

        else:
            # if stagant = 50 then Generating new population like starting our program again because program failed to find solution
            population.clear()
            population.append(mutated_population)

        # showing the generation number and best solution and current fitness value  after every 20 generations
        if i % 20 == 0:
            print("\n\u2192 "+ "Generation: ", i,"  Fitness of Generation ",i," : ", best_solution.fitness )
            print("Best solution till now: ")
            print_schedule(best_solution)
            constraints_satisfied_print(best_solution, courses, course_allocation)

        print(i, "- Fitness of best solution: ", best_solution.fitness, "\t( Fitness of local best solution: ",
              schedule1.fitness, ")", "\n( Stagnation: ", stagnant, ")")

    return best_solution


#------------------------------------------------------------------------------------------------------------------------
#                                                     MAIN
#------------------------------------------------------------------------------------------------------------------------

# the code will start from here 
def main():
    courses, teachers, course_allocation = read_data()
    # solutions in a population
    population_size = random.randint(50, 200)         
     # itterations
    max_generations = 30    
    crossover_probability = 1
    mutation_probability = 0.5

    print('Population size: ',population_size)
    print('Number of generations: ',max_generations)
    print('Mutation probability: ', mutation_probability)
    print('Crossover probability: ',crossover_probability)
   

    Final_Result = genetic_algo(population_size, max_generations, crossover_probability, mutation_probability, courses, teachers,
                       course_allocation)
    
    # AFTER GETTING FINAL SOLUTION PRINTING IT TOO
    print("\n\n--------------------The FINAL SOLUTION--------------------\n\n")
    print_schedule(Final_Result)
    constraints_satisfied_print(Final_Result, courses, course_allocation)



# Telling python to run main method
if __name__ == "__main__":
    main()


Population size:  138
Number of generations:  30
Mutation probability:  0.5
Crossover probability:  1

→ Generation:  0   Fitness of Generation  0  :  7.833333333333333
Best solution till now: 

Day:  Thursday
ClassRoom: C304
MorningExamCode: MG220    | MorningInvigilator: Behjat Zuhaira      
NoonExamCode: DS3011   | NoonInvigilator: Sehrish Hassan      

ClassRoom: C302
MorningExamCode: CY2012   | MorningInvigilator: Noreen Jamil        
NoonExamCode: MG223    | NoonInvigilator: Shehreyar Rashid    

ClassRoom: C301
MorningExamCode: CS217    | MorningInvigilator: Ameen Chilwan       
NoonExamCode: CS118    | NoonInvigilator: Subhan Ullah        

ClassRoom: C309
MorningExamCode: CY2012   | MorningInvigilator: Muhammad bin Qasim  
NoonExamCode: AI2011   | NoonInvigilator: Farwa Batool        

ClassRoom: C306
MorningExamCode: MG220    | MorningInvigilator: Maheen Arshad       
NoonExamCode: CS211    | NoonInvigilator: Hassan Mustafa      

ClassRoom: C305
MorningExamCode: MG220    | M

21 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  0 )
22 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  1 )
23 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  2 )
24 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  3 )
25 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  4 )
26 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  5 )
27 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  6 )
28 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  7 )
29 - Fitness of best solution:  8.5 	( Fitness of local best solution:  8.5 ) 
( Stagnation:  8 )


--------------------The FINAL SOLUTION--------------------



Day:  Thursday
ClassRoom: C304
MorningExamCode: MG220 